# LSTM 모델을 이용한 텍스트 감성분석

### tensorflow

In [1]:
# 예제 2: LSTM으로 한글 감정 분석 (리뷰 데이터)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import re

# 한글 리뷰 샘플 데이터 (긍정: 1, 부정: 0)
korean_reviews = [
    # 긍정 리뷰
    ("이 영화 정말 재미있어요! 강력 추천합니다.", 1),
    ("배우들 연기가 너무 좋고 스토리도 흥미진진해요.", 1),
    ("완전 대박이에요. 다시 보고 싶은 영화네요.", 1),
    ("감동적인 스토리와 훌륭한 연출이 인상적이었습니다.", 1),
    ("최고의 영화 중 하나입니다. 꼭 보세요!", 1),
    ("웃음도 감동도 모두 주는 완벽한 작품이에요.", 1),
    ("놀라운 영상미와 깊이 있는 내용에 감탄했어요.", 1),
    ("시간 가는 줄 모르고 봤습니다. 정말 좋아요.", 1),
    ("예상보다 훨씬 재미있고 의미 있는 영화였어요.", 1),
    ("모든 장면이 인상 깊고 기억에 남을 것 같아요.", 1),
    ("훌륭한 음악과 함께 완벽한 하모니를 이루는 작품이에요.", 1),
    ("배우들의 열연과 감독의 연출력이 돋보이는 영화입니다.", 1),

    # 부정 리뷰
    ("정말 지루하고 재미없어요. 시간 낭비였습니다.", 0),
    ("스토리가 뻔하고 예측 가능해서 실망했어요.", 0),
    ("연기가 어색하고 대사도 어색해서 몰입이 안돼요.", 0),
    ("너무 길고 지루해서 중간에 나오고 싶었어요.", 0),
    ("기대했는데 완전 실망이에요. 별로예요.", 0),
    ("스토리 전개가 느리고 재미없어서 졸았어요.", 0),
    ("예고편이 더 재미있었어요. 본편은 실망.", 0),
    ("돈 아까워요. 다른 영화 볼 걸 그랬어요.", 0),
    ("억지스러운 감동과 뻔한 결말이 아쉬워요.", 0),
    ("배우들 연기가 어색하고 스토리도 이상해요.", 0),
    ("기대와 달리 너무 유치하고 재미없었어요.", 0),
    ("시간이 아까운 영화였습니다. 추천하지 않아요.", 0),

    # 추가 긍정 리뷰
    ("마음이 따뜻해지는 좋은 영화였어요.", 1),
    ("가족과 함께 보기 좋은 훌륭한 작품입니다.", 1),
    ("깊은 여운이 남는 의미 있는 영화네요.", 1),
    ("볼 때마다 새로운 감동을 주는 명작이에요.", 1),
    ("연출과 연기 모든 면에서 완벽했습니다.", 1),

    # 추가 부정 리뷰
    ("내용이 너무 무겁고 우울해서 힘들었어요.", 0),
    ("억지로 만든 것 같은 느낌이 강해요.", 0),
    ("캐릭터들이 매력적이지 않고 공감하기 어려워요.", 0),
    ("결말이 너무 허무하고 아쉬웠습니다.", 0),
    ("과장된 연출이 부자연스러워서 실망했어요.", 0)
]

class KoreanSentimentAnalyzer:
    def __init__(self, max_words=10000, max_length=50):
        self.max_words = max_words
        self.max_length = max_length
        self.tokenizer = Tokenizer(
            num_words=max_words,
            oov_token="<OOV>",
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'  # 한글 처리를 위해 필터 조정
        )

    def preprocess_korean_text(self, text):
        # 한글 텍스트 전처리
        text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
        text = re.sub(r'\s+', ' ', text)     # 여러 공백을 하나로
        text = text.strip()
        return text

    def prepare_data(self, reviews_data):
        # 텍스트와 라벨 분리 및 전처리
        texts = [self.preprocess_korean_text(review[0]) for review in reviews_data]
        labels = [review[1] for review in reviews_data]

        # 토크나이저 훈련
        self.tokenizer.fit_on_texts(texts)

        # 어휘 정보 출력
        print(f"총 어휘 수: {len(self.tokenizer.word_index)}")
        print("빈도 상위 10개 단어:")
        word_freq = sorted(self.tokenizer.word_counts.items(), key=lambda x: x[1], reverse=True)
        for word, freq in word_freq[:10]:
            print(f"  {word}: {freq}")

        # 텍스트를 시퀀스로 변환
        sequences = self.tokenizer.texts_to_sequences(texts)

        # 패딩 적용
        X = pad_sequences(sequences, maxlen=self.max_length, padding='post', truncating='post')
        y = np.array(labels)

        return X, y, texts

    def build_model(self, embedding_dim=100, lstm_units=64):
        model = Sequential([
            Embedding(self.max_words, embedding_dim, input_length=self.max_length),
            LSTM(lstm_units, dropout=0.3, recurrent_dropout=0.3, return_sequences=True),
            LSTM(lstm_units//2, dropout=0.3, recurrent_dropout=0.3),
            Dense(32, activation='relu'),
            Dropout(0.5),
            Dense(1, activation='sigmoid')
        ])

        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        return model

    def predict_sentiment(self, model, text):
        # 텍스트 전처리
        processed_text = self.preprocess_korean_text(text)
        sequence = self.tokenizer.texts_to_sequences([processed_text])
        padded = pad_sequences(sequence, maxlen=self.max_length, padding='post', truncating='post')

        # 예측
        prediction = model.predict(padded, verbose=0)[0][0]

        if prediction > 0.5:
            sentiment = "긍정"
            confidence = prediction
        else:
            sentiment = "부정"
            confidence = 1 - prediction

        return sentiment, confidence, prediction

def run_korean_sentiment_example():
    print("=== 예제 2: LSTM으로 한글 감정 분석 ===\n")

    # 1. 데이터 준비
    analyzer = KoreanSentimentAnalyzer(max_words=5000, max_length=50)
    X, y, texts = analyzer.prepare_data(korean_reviews)

    print(f"\n총 리뷰 수: {len(X)}")
    print(f"긍정 리뷰: {sum(y)}개")
    print(f"부정 리뷰: {len(y) - sum(y)}개")
    print(f"최대 시퀀스 길이: {analyzer.max_length}")

    # 2. 훈련/테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # 3. 모델 구축
    model = analyzer.build_model()
    model.summary()

    # 4. 모델 훈련
    print("\n모델 훈련 중...")
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=16,
        validation_data=(X_test, y_test),
        verbose=1
    )

    # 5. 모델 평가
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\n테스트 정확도: {test_accuracy:.4f}")

    # 상세 분류 리포트
    y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
    print("\n분류 성능 리포트:")
    print(classification_report(y_test, y_pred, target_names=['부정', '긍정']))

    # 6. 새로운 리뷰 감정 분석
    test_reviews = [
        "이 영화 진짜 최고예요! 너무 재미있어서 시간 가는 줄 몰랐어요.",
        "정말 지루하고 재미없는 영화였습니다. 돈이 아까워요.",
        "그럭저럭 볼 만한 영화였어요. 나쁘지 않네요.",
        "스토리가 감동적이고 배우들 연기도 훌륭했습니다.",
        "예상보다 별로였어요. 다른 영화를 볼 걸 그랬네요.",
        "가족과 함께 보기 좋은 따뜻한 영화였습니다."
    ]

    print("\n=== 새로운 리뷰 감정 분석 결과 ===")
    for i, review in enumerate(test_reviews, 1):
        sentiment, confidence, raw_score = analyzer.predict_sentiment(model, review)
        print(f"{i}. 리뷰: '{review}'")
        print(f"   감정: {sentiment} (신뢰도: {confidence:.3f}, 원점수: {raw_score:.3f})\n")

    return model, analyzer, history

# 실행
if __name__ == "__main__":
    model, analyzer, history = run_korean_sentiment_example()

=== 예제 2: LSTM으로 한글 감정 분석 ===

총 어휘 수: 157
빈도 상위 10개 단어:
  너무: 5
  영화: 3
  정말: 3
  연기가: 3
  훌륭한: 3
  있는: 3
  배우들: 2
  스토리도: 2
  완전: 2
  영화네요: 2

총 리뷰 수: 34
긍정 리뷰: 17개
부정 리뷰: 17개
최대 시퀀스 길이: 50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


모델 훈련 중...
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5123 - loss: 0.6928 - val_accuracy: 0.4286 - val_loss: 0.6955
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.4630 - loss: 0.6968 - val_accuracy: 0.4286 - val_loss: 0.6968
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - accuracy: 0.4668 - loss: 0.6936 - val_accuracy: 0.4286 - val_loss: 0.6967
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - accuracy: 0.5826 - loss: 0.6927 - val_accuracy: 0.4286 - val_loss: 0.6973
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - accuracy: 0.5826 - loss: 0.6897 - val_accuracy: 0.4286 - val_loss: 0.6981
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - accuracy: 0.5332 - loss: 0.6865 - val_accuracy: 0.4286 - val_loss: 0.6994
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.5864 - loss: 0.6912 - val_accuracy: 0.4286 - val_loss: 0.6995
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 335ms/step - accuracy: 0.4174 - loss: 0.6875 - val_accuracy: 0.4286 -

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


1. 리뷰: '이 영화 진짜 최고예요! 너무 재미있어서 시간 가는 줄 몰랐어요.'
   감정: 긍정 (신뢰도: 0.508, 원점수: 0.508)

2. 리뷰: '정말 지루하고 재미없는 영화였습니다. 돈이 아까워요.'
   감정: 긍정 (신뢰도: 0.508, 원점수: 0.508)

3. 리뷰: '그럭저럭 볼 만한 영화였어요. 나쁘지 않네요.'
   감정: 긍정 (신뢰도: 0.508, 원점수: 0.508)

4. 리뷰: '스토리가 감동적이고 배우들 연기도 훌륭했습니다.'
   감정: 긍정 (신뢰도: 0.508, 원점수: 0.508)

5. 리뷰: '예상보다 별로였어요. 다른 영화를 볼 걸 그랬네요.'
   감정: 긍정 (신뢰도: 0.508, 원점수: 0.508)

6. 리뷰: '가족과 함께 보기 좋은 따뜻한 영화였습니다.'
   감정: 긍정 (신뢰도: 0.508, 원점수: 0.508)

